# Finance News
from yahoo finance: https://finance.yahoo.com/topic/latest-news/


based on these works
- https://medium.com/@georgemichaeldagogomaynard/web-scraping-dynamic-sites-with-both-selenium-and-beautiful-soup-dim-c679743018de

In [1]:
# Dependencies
# Selenium modules for controlling Chrome browser
from selenium.webdriver import Chrome  # For initializing and controlling the Chrome browser
from selenium import webdriver  # Provides access to the webdriver, allowing interaction with web browsers
from selenium.webdriver.chrome.options import Options  # For configuring Chrome browser options (e.g., headless mode)
from selenium.webdriver.chrome.service import Service  # For managing the ChromeDriver service (e.g., starting, stopping)

# Selenium modules for interacting with web elements
from selenium.webdriver.common.by import By  # For locating elements on a webpage (e.g., By.ID, By.XPATH)
from selenium.webdriver.support.ui import Select  # For interacting with <select> HTML elements (dropdowns)
from selenium.webdriver.support.ui import WebDriverWait  # For implementing explicit waits until a condition is met
from selenium.webdriver.support import expected_conditions as EC  # For defining conditions to wait for (e.g., element visibility)

# Other useful libraries
from fake_useragent import UserAgent  # For generating random user agents to mimic different browsers
import time  # For adding delays (e.g., time.sleep) during the script execution
import requests  # For making HTTP requests to interact with websites directly without using a browser
from bs4 import BeautifulSoup  # For parsing and extracting data from HTML content
import pandas as pd  # For data manipulation, analysis, and creating DataFrames

In [ ]:
# Testing the scraper (replace print with storing data in a structured format later)
import re

news_uri = "https://finance.yahoo.com/topic/latest-news/"

# Use selenium for "dynamic" content loading 
options=Options()
options.add_argument("--headless")  # Run Chrome in headless mode (without GUI)

# Add headers or some bs to mimic a browser request to avoid 429 (Too Many Requests)
ua = UserAgent()
userAgent = ua.random
options.add_argument(f'user-agent={userAgent}')
driver = webdriver.Chrome(options=options)

try: 
    driver.get(news_uri)
    time.sleep(5)  # Wait for the page to load completely
    
    # Scroll to the bottom to load more news
    scroll_count = 3
    for i in range(scroll_count):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Wait for new content to load after scrolling
        print (f"Scrolled {i+1}/{scroll_count} times")
    page = driver.page_source
    news_soup = BeautifulSoup(page, 'html.parser')
except Exception as e:
    print(e)
finally:
    driver.quit()
#print (page.status_code)  # should be 200 if successful
#print (page.content[:100])  


news_list = news_soup.find('ul', class_= "stream-items yf-9xydx9")
news_listings = news_list.find_all('li', class_= "stream-item story-item yf-9xydx9")
print (f"Found {len(news_listings)} news articles")

# TODO: catch None conditions, fix duplicates issue, add ticker percentage change
for news in news_listings:
    # Want to scrape: title, link, publisher, tickers 
    print(news.find('h3').text)  # Article's title
    
    # Use regex to match 'publishing' followed by any other class names
    print(news.find('div', class_=re.compile(r'publishing.*')).text.split('•')[0]) # Publisher name
    
    # Extract tickers and their percentage changes
    if news.find('div', class_='taxonomy-links'):  
        tickers = []
        for ticker in news.find('div', class_='taxonomy-links').find_all('span', class_=re.compile(r'ticker-wrapper.*')):
            # get data from the fin-streamer element
            streamer = ticker.find('fin-streamer', {'data-field': 'regularMarketChangePercent'})
            if streamer:
                print (streamer)
                symbol = streamer.get('data-symbol')
                change = streamer.get_text(strip=True)
                tickers.append({'symbol': symbol, 'change': change})
            else:
                # Fallback if streamer is not found
                tickers.append({'symbol': ticker.text.strip(), 'change': None})
            
        #print (', '.join(tickers))  # All tickers in one line
        print (tickers)
    else:
        print("No tickers found")
    print (news.find('a')['href']) # link to the article
    
    #bug is after here?
    #TODO: make this process faster
    news_detail_url = news.find('a')['href']
    news_detail_page = requests.get(news_detail_url, headers=headers)
    news_detail_soup = BeautifulSoup(news_detail_page.content, 'html.parser')
    
    # writers, time published, article content, 
    # TODO: might use ticker-wrapper from <div class= "slide"> instead
    print(news_detail_soup.find('div', class_=re.compile(r'byline-attr-author.*')).text.strip()) # Authors
    print(news_detail_soup.find('time').text) # Time published
    
    # bug is caused before here?
    
    # Body: for each paragraph in the article content
    body_wrapper = news_detail_soup.find('div', class_='bodyItems-wrapper')
    if body_wrapper:
        for para in body_wrapper.find_all('p'):
            # Check if the paragraph is inside 'read-more-wrapper' to avoid duplication
            if para.find_parent('div', class_='read-more-wrapper'): # if parent is read-more-wrapper
                continue
            print(para.text.strip())
            
    # 'Read more' section
    read_more_wrapper = news_detail_soup.find('div', class_='read-more-wrapper')
    if read_more_wrapper:
        for para in read_more_wrapper.find_all('p'): 
            print(para.text.strip())
    
    
    print ("\n")
# ul and li(class = story-item) tags are used for listing news articles
# For the content of each news article: <div class=content> ->...(below)..
# inside div(class = footer) ->  <div(class = publishing)> for publisher name and time (1st & 2nd text)
# inside div(class = footer) -> <div(class = taxonomy-links)> -> <div(class=ticker-wrapper)> for each ticker

Scrolled 1/3 times
Scrolled 2/3 times
Scrolled 3/3 times
The US auto industry is bracing for an EV winter. Here's why.
Business Insider 
<fin-streamer active="" class="percentChange yf-90gdtp" data-field="regularMarketChangePercent" data-pricehint="2" data-symbol="TAG-USD" data-trend="txt" data-tstyle="default"><span class="d60f3b00">0.00%</span></fin-streamer>
<fin-streamer active="" class="percentChange yf-90gdtp" data-field="regularMarketChangePercent" data-pricehint="2" data-symbol="GAS-USD" data-trend="txt" data-tstyle="default"><span class="d60f3b00 d5882186">-1.85%</span></fin-streamer>
[{'symbol': 'TAG-USD', 'change': '0.00%'}, {'symbol': 'GAS-USD', 'change': '-1.85%'}]
https://finance.yahoo.com/news/us-auto-industry-bracing-ev-090502821.html
Tom Carter
Sat, November 29, 2025 at 4:05 PM GMT+7
EVs were once the future in the US. Now, they're at risk of short-circuiting.
Policy changes, tariffs, and supply chain disruptions have sparked warnings of an EV deep freeze.
One analyst 

In [23]:
soup.prettify()[:200] 

'<!DOCTYPE html>\n<html class="smartphone neo-green dock-upscale" data-color-scheme="auto" data-color-theme-enabled="true" lang="en-US" theme="auto">\n <head>\n  <meta charset="utf-8"/>\n  <meta content="g'

In [8]:
# Connect to MongoDB
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import os
from dotenv import load_dotenv

load_dotenv()
db_username = os.getenv("db_username")
db_password = os.getenv("db_password")
db_host = os.getenv("db_host")

if not all([db_username, db_password, db_host]):
    print("Please set db_username, db_password, and db_host in your .env file")
else:
    uri = f"mongodb+srv://{db_username}:{db_password}@{db_host}"
    client = MongoClient(uri, server_api=ServerApi('1'))

    try:
        client.admin.command('ping')
        print("Pinged your deployment. You successfully connected to MongoDB!")
        
        # Create/Select database and collection
        db = client["finance_news_db"] 
        collection = db["news_articles"]
        print(f"Connected to database: {db.name}, collection: {collection.name}")
        
    except Exception as e:
        print(e)

Pinged your deployment. You successfully connected to MongoDB!
Connected to database: finance_news_db, collection: sample_articles


# Real Scraper w/connection to MongoDB

In [ ]:
# Working scraper!
import re

news_uri = "https://finance.yahoo.com/topic/latest-news/"

# We use Selenium because the 'fin-streamer' percentages are loaded via JavaScript
options = Options()
options.add_argument("--headless") 
ua = UserAgent()
userAgent = ua.random
options.add_argument(f'user-agent={userAgent}')
driver = webdriver.Chrome(options=options)

# Headers for the requests library (used later for article details)
headers = {'User-Agent': userAgent}

try:
    print("Fetching news list via Selenium...")
    driver.get(news_uri)
    time.sleep(3) # Wait for JS to load tickers
    
    # 2. Scroll to load more news
    scroll_count = 3 
    for i in range(scroll_count):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2) # Wait for new items to load
        print(f"Scrolled {i+1}/{scroll_count}")
        
    page_source = driver.page_source
finally:
    driver.quit()

news_soup = BeautifulSoup(page_source, 'html.parser')

news_list = news_soup.find('ul', class_= "stream-items yf-9xydx9")
news_listings = news_list.find_all('li', class_= "stream-item story-item yf-9xydx9")
print(f"Found {len(news_listings)} articles.")

for news in news_listings:
    news_data = {}
    
    # Title
    title = news.find('h3')
    if not title: continue # Skip if malformed
    news_data['title'] = title.text
    
    # Publisher
    publisher = news.find('div', class_=re.compile(r'publishing.*'))
    news_data['publisher'] = publisher.text.split('•')[0] if publisher else "Unknown"
    
    # Tickers & Percentages
    tickers = []
    if news.find('div', class_='taxonomy-links'):  
        for ticker in news.find('div', class_='taxonomy-links').find_all('span', class_=re.compile(r'ticker-wrapper.*')):
            # Find the fin-streamer element for percentage change
            streamer = ticker.find('fin-streamer', {'data-field': 'regularMarketChangePercent'})
            
            if streamer:
                symbol = streamer.get('data-symbol')
                # get_text(strip=True) gets text from inside the nested <span>
                change = streamer.get_text(strip=True) 
                tickers.append({'symbol': symbol, 'change': change})
            else:
                tickers.append({'symbol': ticker.text.strip(), 'change': None})
                
    news_data['tickers'] = tickers
    
    # Link
    link = news.find('a')
    if not link: continue
    news_data['link'] = link['href']
    
    # --- Fetch Article Details (using requests for speed, assuming body is static) ---
    try:
        news_detail_url = news_data['link']
        news_detail_page = requests.get(news_detail_url, headers=headers)
        news_detail_soup = BeautifulSoup(news_detail_page.content, 'html.parser')
        
        # Author
        author = news_detail_soup.find('div', class_=re.compile(r'byline-attr-author.*'))
        news_data['authors'] = author.text.strip() if author else "Unknown"

        # Time
        time = news_detail_soup.find('time')
        news_data['time_published'] = time.text if time else "Unknown"
        
        # Content
        content_text = []
        body_wrapper = news_detail_soup.find('div', class_='bodyItems-wrapper')
        seen_texts = set()
        
        if body_wrapper:
            for para in body_wrapper.find_all('p'):
                text = para.text.strip()
                if not text or text in seen_texts: continue
                if "Most Read from" in text or "Recommended Stories" in text: break
                if para.find_parent('div', class_='read-more-wrapper'): continue
                
                content_text.append(text)
                seen_texts.add(text)
                
        news_data['content'] = "\n".join(content_text)
        
        # Insert into MongoDB
        if 'collection' in globals():
            if collection.count_documents({'link': news_data['link']}) == 0:
                collection.insert_one(news_data)
                print(f"Inserted: {news_data['title']}")
            else:
                print(f"Skipped: {news_data['title']}")
        else:
            print(f"Scraped (DB not connected): {news_data['title']}")
            
    except Exception as e:
        print(f"Error scraping details for {news_data['title']}: {e}")
    
    print ("\n")

Inserted: Micron to invest $9.6 billion in Japan to build AI memory chip plant, Nikkei reports


Inserted: ‘Does Reeves have a plan for growth?’ Your Budget questions answered


Inserted: VC Kara Nortman bet early on women’s sports, and now she’s creating the market


Inserted: Could the EU’s frozen-assets plan really destabilise European bond markets?


Inserted: Ryanair scraps VIP scheme after customers take too many cheap flights


Inserted: This African nation built its development on diamonds. Now it's crashing down


Inserted: A couple earning six figures from 28 rental units explains how they use a 'buy box' to ensure positive cash flow


Inserted: Michael Burry just started a massive group chat on Substack, and it's as chaotic as you'd expect


Inserted: Shoppers are on pace to break Black Friday online spending records and use AI more than ever as sales hit $8.6 billion


Inserted: 3 Highest-Yielding Dividend Kings To Buy, Hold, and Forget


Inserted: Tesla loses some AI staff